# Zeeguu Architecture Reconstruction with Pyreverse

This notebook demonstrates how to use Pyreverse (provided with Pylint) to perform architecture reconstruction on the Zeeguu project. We will generate UML diagrams for both the backend (Zeeguu API) and the React-based frontend.

## Overview

- **Zeeguu API (backend):** located in `data/api`
- **Zeeguu React Frontend:** located in `data/frontend`

Before proceeding, ensure that Pylint (which includes Pyreverse) is installed, and that Graphviz is installed and configured on your system (its bin folder must be in your PATH).

In [1]:
import os
import sys
import subprocess

# Set working directory to the notebook directory (if available)
try:
    notebook_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    notebook_dir = os.getcwd()

os.chdir(notebook_dir)
print(f"Working directory set to: {os.getcwd()}")

# Install required packages using the current Python interpreter
def install_package(package):
    try:
        __import__(package.replace('-', '_'))
        print(f"{package} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_package('pylint')
install_package('graphviz')
install_package('pydot')

# Check Pyreverse version
try:
    subprocess.check_call(["pyreverse", "--version"])
except Exception as e:
    print("Error calling pyreverse. Ensure Pylint is installed and Graphviz is in your PATH.")

## 1. Generate Basic UML Diagrams

We will now generate basic UML diagrams for the API (backend) and the Frontend. The output files will be stored in an output directory.

In [2]:
# Create the output directory using Python (cross-platform)
output_dir = os.path.join('output', 'pyreverse')
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory ensured at: {os.path.abspath(output_dir)}")

In [3]:
# Generate diagrams for the API (backend)
subprocess.check_call([
    "pyreverse", "data/api", "-o", "png", "-p", "zeeguu_api", "-d", output_dir
])

# List generated API files
print(subprocess.getoutput(f"dir {os.path.join(output_dir, 'classes_zeeguu_api.png')}"))

In [4]:
# Generate diagrams for the Frontend
subprocess.check_call([
    "pyreverse", "data/frontend", "-o", "png", "-p", "zeeguu_frontend", "-d", output_dir
])

# List generated Frontend files
print(subprocess.getoutput(f"dir {os.path.join(output_dir, 'classes_zeeguu_frontend.png')}"))

## 2. Displaying the Generated Diagrams

Use IPython’s display functions to show the generated diagrams directly in the notebook.

In [5]:
from IPython.display import Image, display

print("API Classes Diagram:")
display(Image(filename=os.path.join(output_dir, 'classes_zeeguu_api.png')))

print("API Packages Diagram:")
display(Image(filename=os.path.join(output_dir, 'packages_zeeguu_api.png')))

In [6]:
print("Frontend Classes Diagram:")
display(Image(filename=os.path.join(output_dir, 'classes_zeeguu_frontend.png')))

print("Frontend Packages Diagram:")
display(Image(filename=os.path.join(output_dir, 'packages_zeeguu_frontend.png')))

## 3. Advanced Pyreverse Options

Pyreverse offers several advanced options to refine the generated diagrams. Run the help command below to see all options available:

In [7]:
subprocess.check_call(["pyreverse", "--help"])

## 4. Focused Analysis and Filtering

You can generate more focused diagrams by filtering specific modules. For example, below we list some key modules from the API.

In [8]:
# List key API modules (this uses shell commands, adjust for your Windows shell if needed)
print(subprocess.getoutput('dir /B /S data\api\*.py'))

In [9]:
# Generate a focused diagram for core modules
subprocess.check_call([
    "pyreverse", "data/api/zeeguu", "-o", "png", "-p", "zeeguu_core_modules", "-d", output_dir, "--only-classname"
])

print("Core Modules Class Diagram:")
display(Image(filename=os.path.join(output_dir, 'classes_zeeguu_core_modules.png')))

## 5. Enhanced Diagrams with Filtering Options

Below we use additional Pyreverse filtering options (ignoring test directories and private modules, showing ancestors and associations) to generate cleaner diagrams.

In [10]:
# Generate a filtered diagram for the API
subprocess.check_call([
    "pyreverse", "data/api", "-o", "png", "-p", "zeeguu_api_filtered",
    "-d", output_dir,
    "--only-classname",
    "--ignore=test,tests,__pycache__",
    "--show-ancestors",
    "--show-associated",
    "--module-names", "y"
])

print("Filtered API Class Diagram:")
display(Image(filename=os.path.join(output_dir, 'classes_zeeguu_api_filtered.png')))

## 6. Subsystem Analysis with Pyreverse

This section focuses on analyzing specific subsystems of the Zeeguu architecture by isolating parts of the code base. We define a helper function to generate and display diagrams for a given subsystem.

In [11]:
from IPython.display import Image, display

def analyze_subsystem(path, name):
    """Generate and display diagrams for a specific subsystem"""
    if not os.path.exists(path):
        print(f"Path {path} does not exist!")
        return
    
    print(f"\nAnalyzing subsystem: {name}")
    
    # Generate diagrams for the subsystem
    subprocess.check_call([
        "pyreverse", path, "-o", "png", "-p", name, "-d", output_dir,
        "--only-classname",
        "--module-names", "y",
        "--show-ancestors",
        "--show-associated"
    ])
    
    # Display the generated diagrams
    class_diagram = os.path.join(output_dir, f"classes_{name}.png")
    package_diagram = os.path.join(output_dir, f"packages_{name}.png")
    
    if os.path.exists(class_diagram):
        print(f"\n{name} Classes Diagram:")
        display(Image(filename=class_diagram))
    else:
        print(f"No class diagram generated for {name}")
    
    if os.path.exists(package_diagram):
        print(f"\n{name} Packages Diagram:")
        display(Image(filename=package_diagram))
    else:
        print(f"No package diagram generated for {name}")

# Analyze selected API subsystems (adjust paths as needed)
analyze_subsystem(os.path.join('data', 'api', 'zeeguu', 'model'), "zeeguu_model")
analyze_subsystem(os.path.join('data', 'api', 'zeeguu', 'api'), "zeeguu_api_endpoints")
analyze_subsystem(os.path.join('data', 'api', 'zeeguu', 'core'), "zeeguu_core")

## 7. Frontend Subsystem Analysis

Similarly, we analyze key subsystems in the Frontend.

In [12]:
# List frontend subsystems
print(subprocess.getoutput('dir /B /S data\frontend')),

# Analyze selected Frontend subsystems
analyze_subsystem(os.path.join('data', 'frontend', 'src', 'components'), "frontend_components")
analyze_subsystem(os.path.join('data', 'frontend', 'src', 'pages'), "frontend_pages")
analyze_subsystem(os.path.join('data', 'frontend', 'src', 'utils'), "frontend_utils")

## 8. Customizing DOT Files for Enhanced Visualization

Pyreverse produces DOT files that can be further customized. The following function adds custom styling (labels, colors, fonts, etc.) and then converts the DOT file into a PNG image.

In [13]:
def customize_dot_file(input_dot, output_dot, title, node_color="#ADD8E6", edge_color="#4682B4"):
    """Customize a DOT file to improve visualization"""
    if not os.path.exists(input_dot):
        print(f"Input DOT file {input_dot} not found")
        return None
    
    with open(input_dot, 'r') as f:
        dot_content = f.read()
    
    import re
    digraph_pattern = r'(digraph\s+[^{]+\{)'
    styling = (
        f"\n  label=\"Architecture: {title}\";\n"
        "  fontname=\"Arial\";\n"
        "  fontsize=20;\n"
        "  labelloc=\"t\";\n"
        "  bgcolor=\"white\";\n"
        f"  node [shape=box, style=filled, fillcolor=\"{node_color}\", fontname=\"Arial\", fontsize=12];\n"
        f"  edge [color=\"{edge_color}\", penwidth=1.0, fontname=\"Arial\", fontsize=10];\n"
    )
    modified_content = re.sub(digraph_pattern, f'\1{styling}', dot_content)
    
    with open(output_dot, 'w') as f:
        f.write(modified_content)
    
    output_png = output_dot.replace('.dot', '.png')
    subprocess.check_call(["dot", "-Tpng", output_dot, "-o", output_png])
    
    return output_png

## 9. Creating Customized Architecture Views

Below are helper functions that create specialized module views and a layered architecture view, as well as a metrics-enriched view using analysis from the DOT files.

In [14]:
def create_module_view(module_path, output_name, title, ignore_patterns="test,__pycache__"):
    """Create a module viewpoint for a specific part of the system"""
    if not os.path.exists(module_path):
        print(f"Module path {module_path} does not exist")
        return None
    
    custom_dir = os.path.join(output_dir, 'custom')
    os.makedirs(custom_dir, exist_ok=True)
    
    dot_file = os.path.join(custom_dir, f"packages_{output_name}.dot")
    
    subprocess.check_call([
        "pyreverse", module_path, "-o", "dot", "-p", output_name, "-d", custom_dir,
        "--ignore=" + ignore_patterns,
        "--module-names", "y",
        "--only-classname"
    ])
    
    output_custom_dot = os.path.join(custom_dir, f"{output_name}_custom.dot")
    output_png = customize_dot_file(dot_file, output_custom_dot, title, node_color="#E8F5E9", edge_color="#2E7D32")
    
    if output_png and os.path.exists(output_png):
        print(f"\n{title} Viewpoint:")
        display(Image(filename=output_png))
        return output_png
    else:
        print("Failed to generate module view")
        return None

# Create module views for core API functionality and API models
create_module_view(os.path.join('data', 'api', 'zeeguu', 'core'), "zeeguu_core_view", "Zeeguu Core Components")
create_module_view(os.path.join('data', 'api', 'zeeguu', 'model'), "zeeguu_model_view", "Zeeguu Data Model Components")

In [15]:
def create_layered_view(dot_file_path, output_name, title):
    """Create a layered architecture view from a dot file"""
    if not os.path.exists(dot_file_path):
        print(f"Input DOT file {dot_file_path} not found")
        return None
    
    with open(dot_file_path, 'r') as f:
        dot_content = f.read()
    
    output_dot = os.path.join(os.path.dirname(dot_file_path), f"{output_name}_layered.dot")
    
    # Define layered ranking and dummy nodes
    layers = """
    // Layered architecture ranks
    { rank=source; frontend_components; frontend_pages; }
    { rank=same; api_modules; controllers; }
    { rank=same; core_modules; services; }
    { rank=sink; models; database; }
    
    // Layer labels
    subgraph cluster_presentation { label="Presentation Layer"; style=filled; color="#E3F2FD"; frontend_components; frontend_pages; }
    subgraph cluster_application { label="Application Layer"; style=filled; color="#E8F5E9"; api_modules; controllers; }
    subgraph cluster_domain { label="Domain Layer"; style=filled; color="#FFF3E0"; core_modules; services; }
    subgraph cluster_data { label="Data Layer"; style=filled; color="#F3E5F5"; models; database; }
    """
    dummy_nodes = """
    // Dummy nodes for layer visualization
    frontend_components [label="Frontend Components", style=filled, fillcolor="#90CAF9"];
    frontend_pages [label="Frontend Pages", style=filled, fillcolor="#90CAF9"];
    api_modules [label="API Endpoints", style=filled, fillcolor="#A5D6A7"];
    controllers [label="Controllers", style=filled, fillcolor="#A5D6A7"];
    core_modules [label="Core Modules", style=filled, fillcolor="#FFCC80"];
    services [label="Services", style=filled, fillcolor="#FFCC80"];
    models [label="Models", style=filled, fillcolor="#CE93D8"];
    database [label="Database", style=filled, fillcolor="#CE93D8"];
    """
    
    closing_brace_pos = dot_content.rfind('}')
    if closing_brace_pos == -1:
        print("Invalid DOT file format")
        return None
    
    modified_content = dot_content[:closing_brace_pos] + dummy_nodes + layers + dot_content[closing_brace_pos:]
    
    import re
    digraph_pattern = r'(digraph\s+[^{]+\{)'
    styling = (
        f"\n  label=\"Layered Architecture: {title}\";\n"
        "  fontname=\"Arial\";\n"
        "  fontsize=20;\n"
        "  labelloc=\"t\";\n"
        "  rankdir=TB;\n"
        "  compound=true;\n"
        "  splines=ortho;\n"
        "  node [shape=box, style=filled, fontname=\"Arial\", fontsize=12];\n"
        "  edge [penwidth=1.0, fontname=\"Arial\", fontsize=10];\n"
    )
    modified_content = re.sub(digraph_pattern, f'\1{styling}', modified_content)
    
    with open(output_dot, 'w') as f:
        f.write(modified_content)
    
    output_png = output_dot.replace('.dot', '.png')
    subprocess.check_call(["dot", "-Tpng", output_dot, "-o", output_png])
    
    if os.path.exists(output_png):
        print(f"\n{title} Layered Architecture View:")
        display(Image(filename=output_png))
        return output_png
    else:
        print("Failed to generate layered view")
        return None

# Create a layered view from the API package diagram
dot_api = os.path.join(output_dir, 'packages_zeeguu_api_dot.dot')
if not os.path.exists(dot_api):
    subprocess.check_call([
        "pyreverse", "data/api", "-o", "dot", "-p", "zeeguu_api_dot", "-d", output_dir
    ])

create_layered_view(dot_api, "zeeguu_architecture", "Zeeguu System Architecture")

## 10. Adding Metrics Information to the Views

Finally, we can analyze the dependency graph from a DOT file using NetworkX and pydot to compute metrics (like node count, edge count, and density) and incorporate them into the diagram.

In [16]:
import glob
import pydot
import networkx as nx

def analyze_dependencies(dot_file):
    """Analyze module dependencies from a DOT file"""
    if not os.path.exists(dot_file):
        print(f"DOT file {dot_file} not found")
        return {}
    try:
        graphs = pydot.graph_from_dot_file(dot_file)
        if not graphs:
            print("No graph found in DOT file")
            return {}
        graph = graphs[0]
        nx_graph = nx.DiGraph()
        for node in graph.get_nodes():
            name = node.get_name().strip('"')
            nx_graph.add_node(name)
        for edge in graph.get_edges():
            source = edge.get_source().strip('"')
            dest = edge.get_destination().strip('"')
            nx_graph.add_edge(source, dest)
        metrics = {
            'node_count': nx_graph.number_of_nodes(),
            'edge_count': nx_graph.number_of_edges(),
            'density': nx.density(nx_graph),
            'in_degree': dict(nx_graph.in_degree()),
            'out_degree': dict(nx_graph.out_degree()),
            'has_cycles': len(list(nx.simple_cycles(nx_graph))) > 0
        }
        return metrics
    except Exception as e:
        print(f"Error analyzing dependencies: {e}")
        return {}

def create_metrics_enriched_view(dot_file, output_name, title):
    """Create an architectural view enriched with metrics"""
    metrics = analyze_dependencies(dot_file)
    if not metrics:
        print("Failed to analyze metrics")
        return None
    most_depended = sorted(metrics.get('in_degree', {}).items(), key=lambda x: x[1], reverse=True)[:5]
    most_dependent = sorted(metrics.get('out_degree', {}).items(), key=lambda x: x[1], reverse=True)[:5]
    with open(dot_file, 'r') as f:
        dot_content = f.read()
    metrics_label = (
        "  label=<<TABLE BORDER=\"0\" CELLBORDER=\"1\" CELLSPACING=\"0\" CELLPADDING=\"4\">\n"
        f"    <TR><TD COLSPAN=2><B>{title} - Architectural Metrics</B></TD></TR>\n"
        f"    <TR><TD>Modules</TD><TD>{metrics['node_count']}</TD></TR>\n"
        f"    <TR><TD>Dependencies</TD><TD>{metrics['edge_count']}</TD></TR>\n"
        f"    <TR><TD>Density</TD><TD>{metrics['density']:.4f}</TD></TR>\n"
        f"    <TR><TD>Circular Dependencies</TD><TD>{'Yes' if metrics['has_cycles'] else 'No'}</TD></TR>\n"
        "  </TABLE>>;\n  labelloc=\"t\";\n"
    )
    import re
    digraph_pattern = r'(digraph\s+[^{]+\{)'
    styling = (
        "\n  fontname=\"Arial\";\n"
        "  node [shape=box, style=filled, fillcolor=\"#E1F5FE\", fontname=\"Arial\", fontsize=10];\n"
        "  edge [color=\"#0288D1\", penwidth=1.0, fontname=\"Arial\", fontsize=8];\n"
        + metrics_label
    )
    modified_content = re.sub(digraph_pattern, f'\1{styling}', dot_content)
    output_dot = os.path.join(os.path.dirname(dot_file), f"{output_name}_metrics.dot")
    with open(output_dot, 'w') as f:
        f.write(modified_content)
    output_png = output_dot.replace('.dot', '.png')
    subprocess.check_call(["dot", "-Tpng", output_dot, "-o", output_png])
    if os.path.exists(output_png):
        print(f"\n{title} Metrics-Enriched View:")
        display(Image(filename=output_png))
        return output_png
    else:
        print("Failed to generate metrics-enriched view")
        return None

# Ensure the DOT file for the API exists
dot_api = os.path.join(output_dir, 'packages_zeeguu_api_dot.dot')
if not os.path.exists(dot_api):
    subprocess.check_call([
        "pyreverse", "data/api", "-o", "dot", "-p", "zeeguu_api_dot", "-d", output_dir
    ])

create_metrics_enriched_view(dot_api, "zeeguu_api_metrics", "Zeeguu API Architecture")